# Example Usage for Symmetric Group Equivariant Linear Layers

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

import sys
sys.path.append('../')

import nn.traintest as traintest
import nn.symmequiv as symmequiv
from nn.lib import symmpartitions

In [2]:
import importlib
importlib.reload(symmequiv)
importlib.reload(symmpartitions)

<module 'nn.lib.symmpartitions' from '/Users/Edward/Documents/symmetricNNs/examples/../nn/lib/symmpartitions.py'>

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [4]:
print(device)

mps


### Comparison between Diagram and Orbit Bases

#### Diagram Basis

In [5]:
layer_5_1_1_diag = symmequiv.SymmetricGrpEquivLinear(dim_n = 5, order_k = 1, order_l = 1).to(device)

In [6]:
layer_5_1_1_diag.num_weights

2

In [7]:
for basis_matrix in layer_5_1_1_diag.basis_set_matrices:
    print(basis_matrix)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


#### Orbit Basis

In [8]:
layer_5_1_1_orbit = symmequiv.SymmetricGrpEquivLinear(dim_n = 5, order_k = 1, order_l = 1, diag = False).to(device)

In [9]:
layer_5_1_1_orbit.num_weights

2

In [10]:
for basis_matrix in layer_5_1_1_orbit.basis_set_matrices:
    print(basis_matrix)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1.],
        [1., 1., 0., 1., 1.],
        [1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 0.]])


### Example Class using SymmetricGrpEquivLinear layers

In [11]:
class SymmetricGrpEquivNN(nn.Module):
  def __init__(self, dim_n):
    super().__init__()
    self.n = dim_n
    self.layer1 = nn.Sequential(
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 3, order_l = 2).to(device),
        nn.ReLU(),
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 2, order_l = 2).to(device),
        nn.ReLU(),
        symmequiv.SymmetricGrpEquivLinear(dim_n = self.n, order_k = 2, order_l = 0).to(device)
    )

  def forward(self, x):
    x = self.layer1(x)
    return x

In [12]:
model = SymmetricGrpEquivNN(dim_n = 5).to(device)